# OpenMC models
This notebook contains the code to create and run the OpenMC model of 
a PWR pincell, based on the [example online](https://github.com/openmc-dev/openmc/wiki/Example-Jupyter-Notebooks).
Running the model includes developing one-group cross section data and depletion using 
only UOX or MOX fuel.

In [11]:
import numpy as np
import openmc
import openmc.deplete as od

In [12]:
uox_model = openmc.Model()
mox_model = openmc.Model()
openmc.config['cross_sections'] = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

In [36]:
# Materials
uo2 = openmc.Material(1, "uo2")
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)
uo2.volume = 47.78 # area of circle for fuel
uo2.depletable = True

zirconium = openmc.Material(name="zirconium")
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

puo2 = openmc.Material(4)
puo2.add_nuclide('Pu239', 0.94)
puo2.add_nuclide('Pu240', 0.06)
puo2.add_nuclide('O16', 2.0)
puo2.set_density('g/cm3', 11.5)

# Create the mixture
mox = openmc.Material.mix_materials([uo2, puo2], [0.97, 0.03], 'wo')
mox.volume = 0.4778 # area of circle for fuel
mox.depletable = True

uox_model.Materials = openmc.Materials([uo2, zirconium, water])
uox_model.Materials.export_to_xml()
mox_model.Materials = openmc.Materials([mox, zirconium, water])

/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=4.
  warn(msg, IDWarning)


In [14]:
# Geometry
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_inner_radius = openmc.ZCylinder(r=0.40)
clad_outer_radius = openmc.ZCylinder(r=0.46)

fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

uox_fuel = openmc.Cell(name='fuel')
uox_fuel.fill = uo2
uox_fuel.region = fuel_region

mox_fuel = openmc.Cell(name='fuel')
mox_fuel.fill = mox
mox_fuel.region = fuel_region

gap = openmc.Cell(name='air gap')
gap.region = gap_region

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

pitch = 1.26
box = openmc.model.RectangularPrism(width=pitch, height=pitch,
                               boundary_type='reflective')
water_region = -box & +clad_outer_radius
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

uox_universe = openmc.Universe(cells=(uox_fuel, gap, clad, moderator))
mox_universe = openmc.Universe(cells=(mox_fuel, gap, clad, moderator))

uox_model.geometry = openmc.Geometry(uox_universe)
uox_model.geometry.export_to_xml()
mox_model.geometry = openmc.Geometry(mox_universe)

In [15]:
# Settings
settings = openmc.Settings()
settings.batches = 100
settings.inactive = 10
settings.particles = 1000
settings.output = {'tallies':True}
uox_model.settings = settings
mox_model.settings = settings
uox_model.settings.export_to_xml()

In [38]:
# Generate one-group cross section data
uox_flux_xs = od.get_microxs_and_flux(uox_model, 
                                       domains = [uox_fuel], 
                                       chain_file="chain_endfb71_pwr.xml")
mox_flux_xs = od.get_microxs_and_flux(mox_model, 
                                       domains = [mox_fuel], 
                                       chain_file="chain_endfb71_pwr.xml")

[csi365319:472149] mca_base_component_repository_open: unable to open mca_btl_openib: librdmacm.so.1: cannot open shared object file: No such file or directory (ignored)
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%

In [ ]:
# save cross sections from UOX model for use in Deplete Reactor
uox_flux_xs[1][0].to_csv("micro_xs.csv")
mox_flux_xs[1][0].to_csv("mox_xs.csv")

In [39]:
uox_ind_op = od.IndependentOperator(openmc.Materials([uo2]), 
                                [uox_flux_xs[0][0]*uo2.volume],
                                uox_flux_xs[1],
                                "chain_endfb71_pwr.xml")
mox_ind_op = od.IndependentOperator(openmc.Materials([mox]), 
                                [mox_flux_xs[0][0]*mox.volume],
                                mox_flux_xs[1],
                                "chain_endfb71_pwr.xml")

In [40]:
uox_integrator = od.PredictorIntegrator(uox_ind_op,
                                    [30]*36, # 3 cycles of operation
                                   power_density = 46.3, # linear power density in W/g
                                   timestep_units='d')
uox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=2592000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=5184000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=7776000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=10368000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=12960000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=15552000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=18144000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=20736000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=23328000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=25920000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=28512000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=31104000.0 s, dt=2592000 s, source=19500.60858644912
[openmc.deplete] t=33696000.0 s, dt=2592000 s, source=19500.60858644912
[o

In [41]:
! mv depletion_results.h5 uox_depletion_results.h5

In [42]:
mox_integrator = od.PredictorIntegrator(mox_ind_op,
                                    np.ones(12*3)*30,
                                   power_density = 46.3, # linear power density in W/g
                                   timestep_units='d')
mox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=33696000.0 s, dt

In [43]:
! mv depletion_results.h5 mox_depletion_results.h5

In [44]:
# Print depletion material compositions
uox_results = od.Results("uox_depletion_results.h5")
mox_results = od.Results("mox_depletion_results.h5")

In [45]:
uox_spent = uox_results.export_to_materials(-1)
mox_spent = mox_results.export_to_materials(1)

/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=4.
  warn(msg, IDWarning)
/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=5.
  warn(msg, IDWarning)


In [46]:
#printing in format for Cyclus recipe
total = 0
weight_frac = 0
pu_frac = 0
for nuclide in uox_spent[0].nuclides:
    total += nuclide.percent
    #if nuclide.percent <= 1e-10:
    #    continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent}</comp>  </nuclide>")
    weight_frac += nuclide.percent*A
    if Z == 94:
        pu_frac += nuclide.percent*A
    total += nuclide.percent

<nuclide>  <id>892250000</id> <comp>2.1840938415212524e-20</comp>  </nuclide>
<nuclide>  <id>892260000</id> <comp>7.291199297078441e-23</comp>  </nuclide>
<nuclide>  <id>892270000</id> <comp>2.223925672187054e-18</comp>  </nuclide>
<nuclide>  <id>471070000</id> <comp>2.3363670459643215e-12</comp>  </nuclide>
<nuclide>  <id>471080000</id> <comp>2.978898772156138e-16</comp>  </nuclide>
<nuclide>  <id>471090000</id> <comp>6.538356614946153e-06</comp>  </nuclide>
<nuclide>  <id>471100001</id> <comp>1.281273174859606e-07</comp>  </nuclide>
<nuclide>  <id>471110000</id> <comp>3.9476099470305406e-08</comp>  </nuclide>
<nuclide>  <id>471120000</id> <comp>2.921875996488947e-10</comp>  </nuclide>
<nuclide>  <id>471130000</id> <comp>2.1831898777518985e-10</comp>  </nuclide>
<nuclide>  <id>471140000</id> <comp>5.040149692741383e-14</comp>  </nuclide>
<nuclide>  <id>471150000</id> <comp>8.7576201871381e-12</comp>  </nuclide>
<nuclide>  <id>471160000</id> <comp>1.7208460822691286e-12</comp>  </nucli

In [48]:
#printing in format for Cyclus recipe
for nuclide in mox_spent[0].nuclides:
    if nuclide.percent <= 1e-10:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent}</comp>  </nuclide>")

<nuclide>  <id>922350000</id> <comp>0.03</comp>  </nuclide>
<nuclide>  <id>922380000</id> <comp>0.97</comp>  </nuclide>
<nuclide>  <id>80160000</id> <comp>2.0</comp>  </nuclide>
